In [1]:
#package loading 
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter
from dateutil.relativedelta import relativedelta
from scipy.stats import percentileofscore
import umap
from tqdm import tqdm
import cudf
import pickle
import sys

In [2]:
from utility import *
from preprocess import *

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
death_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/death_cancer.pkl','rb'))

In [10]:
vax_cancer=pickle.load( open('/data/datasets/changyuyin/2_COVIDwithCancer/vax_cancer.pkl','rb'))

In [11]:
all_cancer_pid = pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/pid_dx_cancer_subtypes.pkl','rb'))

# Patient number by cancer subtypesdeath_cancer

## Aggregation by vital status

In [12]:
subtype_summary,all_cancer_pid = cancer_subtype_summary(all_cancer_pid,death_cancer)

Number of patient with only one caner subtype 23171


In [13]:
len(set(all_cancer_pid.loc[all_cancer_pid['IS_ALIVE']=="ALIVE"].ID))

29242

In [14]:
subtype_summary

IS_ALIVE,ALIVE,DIED,N_pat_total,N_pat_MultiCancer,Death_rate(%),MultiCancer_rate(%)
subtype,,,,,,
Cervical,5355,208,5563,1043,3.7,18.7
Secondar cancer,3992,812,4804,4446,16.9,92.5
Other,4665,116,4781,0,2.4,0.0
Skin,4245,424,4669,1929,9.1,41.3
Breast,3956,295,4251,2015,6.9,47.4
Prostate,2306,318,2624,1143,12.1,43.6
Head and neck,1963,229,2192,1243,10.4,56.7
Colorectal,1854,221,2075,1252,10.7,60.3
Lung,1660,364,2024,1453,18.0,71.8


In [15]:
subtype_summary.to_csv('/home/changyu.yin/2.COVIDwithCancer/99_outputs/EDA_subtype_summary.csv')

## Co-occurrence matrix

In [16]:
cooccurrence_matrix = cancer_subtype_cooccurrence(all_cancer_pid)

In [17]:
cooccurrence_matrix

,subtype_Cervical,subtype_Secondar cancer,subtype_Other,subtype_Skin,subtype_Breast,subtype_Prostate,subtype_Head and neck,subtype_Colorectal,subtype_Lung,subtype_Non-Hodgkin's lymphoma,...,subtype_Liver,subtype_Brain,subtype_Ovarian,subtype_Stomach,subtype_Pancretic,subtype_Hodgkin's lymphoma,subtype_Uteral,subtype_Esophageal,subtype_Small intestine,subtype_Testicular
subtype_Cervical,5563,322,0,198,258,34,109,118,106,79,...,35,33,143,17,18,22,125,13,6,3
subtype_Secondar cancer,322,4804,0,613,1047,487,527,723,1061,311,...,311,207,245,145,271,56,107,107,82,39
subtype_Other,0,0,4781,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
subtype_Skin,198,613,0,4669,358,265,435,274,244,174,...,72,61,55,51,29,24,41,33,29,7
subtype_Breast,258,1047,0,358,4251,99,146,172,222,145,...,49,78,84,39,26,31,90,26,31,2
subtype_Prostate,34,487,0,265,99,2624,114,112,108,85,...,37,17,3,30,41,9,11,13,15,9
subtype_Head and neck,109,527,0,435,146,114,2192,73,163,92,...,24,47,25,125,13,21,21,54,86,5
subtype_Colorectal,118,723,0,274,172,112,73,2075,156,70,...,100,32,69,63,51,20,43,22,41,4
subtype_Lung,106,1061,0,244,222,108,163,156,2024,113,...,68,90,45,30,41,28,30,22,17,9
subtype_Non-Hodgkin's lymphoma,79,311,0,174,145,85,92,70,113,1880,...,30,61,20,34,18,271,15,13,14,8


In [18]:
cooccurrence_matrix.to_csv('/home/changyu.yin/2.COVIDwithCancer/99_outputs/EDA_subtype_cooccurrence_matrix.csv')

# Vaccination summary

In [29]:
vax_cancer[['DOMAIN','VX_CODE_TYPE']].value_counts()

DOMAIN        VX_CODE_TYPE
PROCEDURE     CH              5977
PRESCRIBING   RX              1809
DISPENSING    ND              1289
IMMUNIZATION  CX               198
dtype: int64

In [30]:
all_cancer_pid_vax_agg, all_cancer_pid_vax, vax_num_agg = cancer_subtype_vax(all_cancer_pid,vax_cancer)

Number of COVIDwithCancer paitent vaccinated: 3966 out of 31680
Percentage of COVIDwithCancer paitent with vaccination data(%): 12.52


In [31]:
vax_num_agg

,ID,N_vax_record,N_vax_record_cap,N_vax_record_adj,N_vax
0,19Oct2022covidpasc00147983,215,4,2,4
1,19Oct2022covidpasc00050967,10,4,4,4
2,19Oct2022covidpasc00003104,10,4,4,4
3,19Oct2022covidpasc00166383,10,4,4,4
4,19Oct2022covidpasc00046495,9,4,3,4
...,...,...,...,...,...
3961,19Oct2022covidpasc00113090,1,1,1,1
3962,19Oct2022covidpasc00112929,1,1,1,1
3963,19Oct2022covidpasc00112710,1,1,1,1
3964,19Oct2022covidpasc00112577,1,1,1,1


In [17]:
vax_num_agg['N_vax_record'].value_counts()

1      1409
2      1349
4       656
3       329
6       116
5        66
8        18
7        16
10        3
9         3
215       1
Name: N_vax_record, dtype: int64

In [27]:
vax_cancer.loc[vax_cancer['ID']=='19Oct2022covidpasc00046495']

,ID,VX_ID,VX_CODE_TYPE,VX_CODE,VX_DATE,DOMAIN,VX_ID_RAW
1812,19Oct2022covidpasc00046495,19Oct2022covidpasc0004649522849602022-02-02,RX,2284960,2022-02-02,PRESCRIBING,PRESCRIBINGcfhOcr8CS1WrqQC+ZvEfIEF7
1852,19Oct2022covidpasc00046495,19Oct2022covidpasc0004649522849602022-01-31,RX,2284960,2022-01-31,PRESCRIBING,PRESCRIBINGcfhOcr8CSlOtowG8ZvEfIEF7
1853,19Oct2022covidpasc00046495,19Oct2022covidpasc0004649522849602022-02-01,RX,2284960,2022-02-01,PRESCRIBING,PRESCRIBINGcfhOcr8CSlOuqga7ZvEfIEF7
4531,19Oct2022covidpasc00046495,19Oct2022covidpasc000464950011A2021-02-04,CH,0011A,2021-02-04,PROCEDURE,PROCEDUREcfhOcr8LSVKprQC+
4532,19Oct2022covidpasc00046495,19Oct2022covidpasc00046495913012021-02-04,CH,91301,2021-02-04,PROCEDURE,PROCEDUREcfhOcr8LSVKprQC5
5215,19Oct2022covidpasc00046495,19Oct2022covidpasc000464950013A2021-09-01,CH,0013A,2021-09-01,PROCEDURE,PROCEDUREcfhOcrgBTVKorAe/
5461,19Oct2022covidpasc00046495,19Oct2022covidpasc00046495913012021-03-04,CH,91301,2021-03-04,PROCEDURE,PROCEDUREcfhOcrgDSV+uqwO9
5608,19Oct2022covidpasc00046495,19Oct2022covidpasc00046495913012021-09-01,CH,91301,2021-09-01,PROCEDURE,PROCEDUREcfhOcrgBTFarrgW+
6125,19Oct2022covidpasc00046495,19Oct2022covidpasc000464950012A2021-03-04,CH,0012A,2021-03-04,PROCEDURE,PROCEDUREcfhOcrgDT1KgrgW9


In [23]:
#abnormal value check
prescribing_cancer_vax = pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/prescribing_cancer_vax.pkl','rb'))
prescribing_cancer_vax.loc[prescribing_cancer_vax['ID']=='19Oct2022covidpasc00147983'].sort_values(['VX_CODE','VX_DATE'])

,VX_ID,ID,ENCOUNTERID,RX_PROVIDERID,VX_DATE,RX_ORDER_TIME,RX_START_DATE,RX_END_DATE,RX_DOSE_ORDERED,RX_DOSE_ORDERED_UNIT,...,RX_FREQUENCY,RX_PRN_FLAG,RX_ROUTE,RX_BASIS,VX_CODE,RX_SOURCE,RX_DISPENSE_AS_WRITTEN,SOURCE_masked,VX_CODE_TYPE,DOMAIN
210,PRESCRIBINGcfNPd7kLHVStrAGwKfAeJEAs6q4WijB37gd...,19Oct2022covidpasc00147983,cfNPcbgASl+rqwC5,NaN,2021-06-25,00:00,NaN,NaN,100.0,OT,...,NI,N,OT,NaN,2470234,OD,NaN,source_1,RX,PRESCRIBING
179,PRESCRIBINGcfNPcesBHVOtqVfqfPgaJBUssPZGhj8luAE...,19Oct2022covidpasc00147983,cfNPcbgESVapqAG7,NaN,2021-09-12,00:00,NaN,NaN,0.5,mL,...,NI,N,OT,NaN,2479835,OD,NaN,source_1,RX,PRESCRIBING
208,PRESCRIBINGcfNPdO5TTQX//lO7KaAfJBR8sPYU22QhsFY...,19Oct2022covidpasc00147983,cfNPcbgLS1ehowOx,NaN,2021-09-07,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING
260,PRESCRIBINGcfNPIO4LGlb7/we9L/VNJBd46vcTiDYluFA...,19Oct2022covidpasc00147983,cfNPcbgLS1ehowOx,NaN,2021-09-07,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING
335,PRESCRIBINGCOVcfNPcOoKTQCp/wztf6IfJERy6vcW3GBy...,19Oct2022covidpasc00147983,COVcfNPcbgLS1ehowOx,NaN,2021-09-07,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING
375,PRESCRIBINGCOVcfNPIrlTSVOg/la6KPQWJBAova4TiDd6...,19Oct2022covidpasc00147983,COVcfNPcbgLS1ehowOx,NaN,2021-09-07,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING
3107,PRESCRIBINGcfNPIbtUQ177+AW7ffcbJEMu6K5Aiz5w7gp...,19Oct2022covidpasc00147983,cfNPcbgLS1ehowOx,cfNPLelc,2021-09-07,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING
3145,PRESCRIBINGcfNPIrpRHl6hqwy8fPcbJBArsa5Fj2d7vwE...,19Oct2022covidpasc00147983,cfNPcbgLS1ehowOx,cfNPLelc,2021-09-07,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING
152,PRESCRIBINGcfNPc+tRSgeo/1Ptc/cXJEN96q0QiWR3ugd...,19Oct2022covidpasc00147983,cfNPcbkCTlStrQ2/,NaN,2021-09-28,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING
291,PRESCRIBINGcfNPIusGH1b/+FTtKaUXJBUr660X32dyuQU...,19Oct2022covidpasc00147983,cfNPcbkCTlStrQ2/,NaN,2021-09-28,00:00,NaN,NaN,1000.0,OT,...,NI,N,OT,NaN,2559909,OD,NaN,source_1,RX,PRESCRIBING


In [24]:
#abnormal value check
procedure_cancer_vax = pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/procedure_cancer_vax.pkl','rb'))
procedure_cancer_vax.loc[procedure_cancer_vax['ID']=='19Oct2022covidpasc00147983'].sort_values(['VX_CODE','VX_DATE'])

,VX_ID,ID,ENCOUNTERID,ENC_TYPE,ADMIT_DATE,PROVIDERID,VX_DATE,VX_CODE,VX_CODE_TYPE,PX_SOURCE,PPX,SOURCE_masked,DOMAIN
948,PROCEDUREcPNHe852TSCp3AGlD4MZIVx+zPllk0cCuQtWJ...,19Oct2022covidpasc00147983,cPNHE7kLTlWuow26fvAcIUg=,AV,2021-02-19,cPNHcrEDSV+uqw26eg==,2021-02-19,0001A,CH,BI,NUL,source_15,PROCEDURE
1106,PROCEDUREcfNPc+sCHwD4/gPqLfIfJBBzuK4X2jYm7VRCA...,19Oct2022covidpasc00147983,cfNPcrEHTFKorwa9,AV,2021-02-26,NaN,2021-02-26,0001A,CH,OD,NI,source_1,PROCEDURE
1186,PROCEDUREcfNPcbBQSFX8+1G4cvdNJEJ5uK0djDdzvgBDA...,19Oct2022covidpasc00147983,cfNPcrEHQ1eoqwG9,AV,2021-02-26,NaN,2021-02-26,0001A,CH,OD,NI,source_1,PROCEDURE
1293,PROCEDUREcfNPcLsFTVWoqgW/efIXJEZ/sK0X3D526QVKB...,19Oct2022covidpasc00147983,cfNPcrEHQ1evqgK7,AV,2021-02-26,NaN,2021-02-26,0001A,CH,OD,NI,source_1,PROCEDURE
1399,PROCEDUREcfNPcrgHGFCv+wbrf/kbJBB66/cWhjB3uApPB...,19Oct2022covidpasc00147983,cfNPcrEHQ1eqqgO7,AV,2021-02-26,NaN,2021-02-26,0001A,CH,OD,NI,source_1,PROCEDURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,PROCEDUREcfNPcL4BHVP7qFOxL/VMJEcpu65G32Jw6VFKU...,19Oct2022covidpasc00147983,cfNPcrEKQlOrqwG7,AV,2021-05-05,NaN,2021-05-05,91301,CH,OD,NI,source_1,PROCEDURE
2007,PROCEDUREcfNPIe1TGgT8qFPsLfQWJBQu7/YUjmAgvwJLU...,19Oct2022covidpasc00147983,cfNPcrEKQlGtowS7,AV,2021-05-05,NaN,2021-05-05,91301,CH,OD,NI,source_1,PROCEDURE
6275,PROCEDUREcfNPe+kBSgKpqALtePNKJBJzvPYShzUlvVFIU...,19Oct2022covidpasc00147983,cfNPcrEAS1Kqqwe9,AV,2021-01-06,cfNPLelc,NaN,91301,CH,OD,NI,source_1,PROCEDURE
6276,PROCEDUREcfNPILAKSwT//gy7fPdMJBQrvq0cjGV0ugpKU...,19Oct2022covidpasc00147983,cfNPcrEASlaqowWx,AV,2021-01-06,cfNPLelc,NaN,91301,CH,OD,NI,source_1,PROCEDURE


In [21]:
vax_num_agg['N_vax_record_cap'].value_counts()

1    1409
2    1349
4     879
3     329
Name: N_vax_record_cap, dtype: int64

In [22]:
vax_num_agg['N_vax'].value_counts()

1    1394
2    1145
4    1025
3     402
Name: N_vax, dtype: int64

In [23]:
all_cancer_pid_vax_agg

,N_pat,N_vax_0,N_vax_1,N_vax_2,N_vax_3,N_vax_noless4,N_vaxed
subtype,,,,,,,
Cervical,5563,4870,240,219,45,189,693
Secondar cancer,4804,4195,241,142,68,158,609
Other,4781,4291,119,248,49,74,490
Skin,4669,3882,249,173,72,293,787
Breast,4251,3734,182,131,58,146,517
Prostate,2624,2295,115,70,30,114,329
Head and neck,2192,1844,94,128,57,69,348
Colorectal,2075,1822,96,56,32,69,253
Lung,2024,1737,123,67,28,69,287


In [24]:
all_cancer_pid_vax_agg.to_csv('/home/changyu.yin/2.COVIDwithCancer/99_outputs/EDA_N_vax_by_subtype.csv')